# PWMの値の設定

## Boardの判定

In [1]:
import Jetson.GPIO as GPIO

BOARD_NAME=GPIO.gpio_pin_data.get_data()[0]
if BOARD_NAME == "JETSON_NX":
    print("Jetson Xavier NXを認識")
    I2C_BUSNUM = 8
elif BOARD_NAME == "JETSON_XAVIER":
    print("Jetson AGX Xavierを認識")
    I2C_BUSNUM = 8
elif BOARD_NAME == "JETSON_NANO":
    print("Jetson NANOを認識")
    I2C_BUSNUM = 1

Jetson NANOを認識


## I2Cの認識確認

0x08: PWMの値の吸い上げ<br>
0x3c: OLED　Display<br>
0x40: PCA9685(PWMの出力)<br>
0x70: PCA9685(PWMの出力)<br>

In [2]:
!i2cdetect -y -r $I2C_BUSNUM

     0  1  2  3  4  5  6  7  8  9  a  b  c  d  e  f
00:          -- -- -- -- -- -- -- -- -- -- -- -- -- 
10: -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- 
20: -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- 
30: -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- 
40: 40 -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- 
50: -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- 
60: -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- 
70: -- -- -- -- -- -- -- --                         


## ラジコンの制御

Jetson Nano 2G, 4GはBUSNUMを1に指定します。<br>
Jetson NXは、BUSNUMを8に指定します。<br>

タミヤのラジコンは60Hzを指定して制御します。<br>

In [3]:
import Fabo_PCA9685
import time
import pkg_resources
import smbus
import time
import json

INITIAL_VALUE = 410
pwm_center = INITIAL_VALUE
pwm_right = INITIAL_VALUE + 100
pwm_left = INITIAL_VALUE - 100
pwm_stop = INITIAL_VALUE

try:
    with open('pwm_params.json') as f:
        json_str = json.load(f)

        pwm_stop = json_str["pwm_speed"]["stop"]
        pwm_left = json_str["pwm_steering"]["left"]
        pwm_center = json_str["pwm_steering"]["center"]
        pwm_right = json_str["pwm_steering"]["right"]
except:
    print("New setting")


SMBUS='smbus'
BUSNUM=I2C_BUSNUM
SERVO_HZ=60
bus = smbus.SMBus(BUSNUM)
PCA9685 = Fabo_PCA9685.PCA9685(bus,INITIAL_VALUE,address=0x40)
PCA9685.set_hz(SERVO_HZ)

New setting


## ハンドルのPWM値の設定

In [4]:
from ipywidgets import Button, Layout, Textarea, HBox, VBox
import ipywidgets
import sys
import threading
import time

STEERING_CH = 0
THROTTLE_CH = 1

center_widget = ipywidgets.IntText(description='中央', value=pwm_center)
right_widget = ipywidgets.IntText(description='右', value=pwm_left)
left_widget = ipywidgets.IntText(description='左', value=pwm_right)
size_slider = ipywidgets.IntSlider(description='振幅', min=0, max=150, step=1, value=100, orientation='horizontal')
check_button = ipywidgets.Button(description='Check Center')
check_left_button = ipywidgets.Button(description='Check Left(左)')
check_right_button = ipywidgets.Button(description='Check Right(右)')

check_box = ipywidgets.Checkbox(False, description='Reserve')

pwm_left = 0
pwm_center = 0
pwm_right = 0

reserve = 0

def check_center(c):
    global pwm_center,center_widget
    pwm_center = center_widget.value
    PCA9685.set_channel_value(STEERING_CH, center_widget.value)

def check_left(c):
    global pwm_left,left_widget
    pwm_left = left_widget.value
    PCA9685.set_channel_value(STEERING_CH, pwm_left)
    
def check_right(c):
    global pwm_right,right_widget
    pwm_right = right_widget.value
    PCA9685.set_channel_value(STEERING_CH, pwm_right)
    
def check_left_right(c):
    global pwm_left, pwm_center, pwm_right
    pwm_left = left_widget.value
    PCA9685.set_channel_value(STEERING_CH, pwm_left)
    time.sleep(1)
    pwm_right = right_widget.value
    PCA9685.set_channel_value(STEERING_CH, pwm_right)
    time.sleep(1)
    pwm_center = center_widget.value
    PCA9685.set_channel_value(STEERING_CH, center_widget.value)
    
def on_value_change(change):
    global reserve, pwm_left, pwm_center, pwm_right
    if reserve == 0:
        pwm_left = int(center_widget.value - size_slider.value)
        pwm_right = int(center_widget.value + size_slider.value)
    elif reserve == 1:
        pwm_left = int(center_widget.value + size_slider.value)
        pwm_right = int(center_widget.value - size_slider.value)
    
    left_widget.value = pwm_left
    right_widget.value = pwm_right

def on_reserve(change):
    global reserve, pwm_left, pwm_center, pwm_right
    reserve^=True
    if reserve == 0:
        pwm_left = int(center_widget.value - size_slider.value)
        pwm_right = int(center_widget.value + size_slider.value)
    elif reserve == 1:
        pwm_left = int(center_widget.value + size_slider.value)
        pwm_right = int(center_widget.value - size_slider.value)
    
    left_widget.value = pwm_left
    right_widget.value = pwm_right
    
center_widget.observe(on_value_change)
size_slider.observe(on_value_change)
check_box.observe(on_reserve)
check_button.on_click(check_center)
check_left_button.on_click(check_left)
check_right_button.on_click(check_right)
left_widget.value = center_widget.value - size_slider.value
right_widget.value = center_widget.value + size_slider.value

In [5]:
check_steering_widget = ipywidgets.VBox([
    ipywidgets.HBox([center_widget, check_button]),
    ipywidgets.HBox([size_slider,check_box]),
    ipywidgets.HBox([left_widget,check_left_button,right_widget,check_right_button]),
])

display(check_steering_widget)

## 前進・後進・停止の値の設定

In [6]:
stop_widget = ipywidgets.IntText(description='停止', value=INITIAL_VALUE)
check_button = ipywidgets.Button(description='Set Stop')
front_slider = ipywidgets.IntSlider(description='前進幅', min=-100, max=100, step=1, value=0, orientation='horizontal')
front_widget = ipywidgets.IntText(description='前進', value=pwm_stop)
back_slider = ipywidgets.IntSlider(description='後進幅', min=-100, max=100, step=1, value=0, orientation='horizontal')
back_widget = ipywidgets.IntText(description='後進', value=pwm_stop)
stop_button = ipywidgets.Button(description='Stop')
set_front_button = ipywidgets.Button(description='Set front')
set_back_button = ipywidgets.Button(description='Set back')

In [7]:
def set_stop(c):
    global stop_widget,pwm_stop,front_slider, back_slider
    pwm_stop = stop_widget.value
    front_widget.value = stop_widget.value + front_slider.value 
    back_widget.value = stop_widget.value + back_slider.value
    PCA9685.set_channel_value(THROTTLE_CH, pwm_stop)
    time.sleep(0.1)
    PCA9685.set_channel_value(THROTTLE_CH, pwm_stop)
    
def set_front(change):
    global stop_widget, front_slider, pwm_front
    pwm_front = stop_widget.value + front_slider.value
    PCA9685.set_channel_value(THROTTLE_CH, pwm_front)

def set_back(change):
    global stop_widget, back_slider, pwm_back
    pwm_stop = stop_widget.value
    PCA9685.set_channel_value(THROTTLE_CH, pwm_stop)
    time.sleep(0.1)
    PCA9685.set_channel_value(THROTTLE_CH, pwm_stop)
    time.sleep(0.1)
    pwm_back = stop_widget.value + back_slider.value
    PCA9685.set_channel_value(THROTTLE_CH, pwm_back)
    
def on_front_change(change):
    global front_slider, front_widget, stop_widget
    front_widget.value = stop_widget.value + front_slider.value
    
def on_back_change(change):
    global back_slider, back_widget, stop_widget
    back_widget.value = stop_widget.value + back_slider.value
    
def stop(change):
    global stop_widget,speed_slider,pwm_stop
    PCA9685.set_channel_value(THROTTLE_CH, pwm_stop)
    
front_slider.observe(on_front_change)
back_slider.observe(on_back_change)
check_button.on_click(set_stop)
stop_button.on_click(stop)
set_front_button.on_click(set_front)
set_back_button.on_click(set_back) 
    
check_speed_widget = ipywidgets.VBox([
    ipywidgets.HBox([stop_widget, check_button]),
    ipywidgets.HBox([front_slider,front_widget,set_front_button]),
    ipywidgets.HBox([back_slider,back_widget,set_back_button]),
    ipywidgets.HBox([stop_button]),
])

display(check_speed_widget)

In [8]:
pwm_front = front_widget.value
pwm_stop = stop_widget.value
pwm_back = back_widget.value

print("raw 前進: %s" % pwm_front)
print("raw 停止: %s" % pwm_stop)
print("raw 後進: %s" % pwm_back)

raw 前進: 410
raw 停止: 410
raw 後進: 410


In [9]:
import json

save_button = ipywidgets.Button(description='Save')

def save(change):
    global left_widget, center_widget, right_widget, front_widget, stop_widget, back_widget
    json_str = {
        "pwm_steering": {
            "left": left_widget.value,
            "center": center_widget.value,
            "right": right_widget.value,
        },
        "pwm_speed": {
            "front": front_widget.value,
            "stop": stop_widget.value,
            "back": back_widget.value,
        }
    }

    with open('pwm_params.json', 'w') as f:
        json.dump(json_str, f, ensure_ascii=False)


save_button.on_click(save)
        
check_value_widget = ipywidgets.VBox([
    ipywidgets.HBox([left_widget, center_widget, right_widget]),
    ipywidgets.HBox([front_widget, stop_widget, back_widget]),
    ipywidgets.HBox([save_button]),
])

display(check_value_widget)

### 確認

設定した値の確認をおこないます。

In [11]:
with open('pwm_params.json') as f:
    json_str = json.load(f)
    
    front = json_str["pwm_speed"]["front"]
    stop = json_str["pwm_speed"]["stop"]
    back = json_str["pwm_speed"]["back"]
    left = json_str["pwm_steering"]["left"]
    center = json_str["pwm_steering"]["center"]
    right = json_str["pwm_steering"]["right"]

In [12]:
check_left_widget = ipywidgets.IntText(description='左', value=left)
check_center_widget = ipywidgets.IntText(description='中央', value=center)
check_right_widget = ipywidgets.IntText(description='右', value=right)
check_front_widget = ipywidgets.IntText(description='前進', value=front)
check_stop_widget = ipywidgets.IntText(description='停止', value=stop)
check_back_widget = ipywidgets.IntText(description='後進', value=back)

stop_button = ipywidgets.Button(description='Stop')
check_center_button = ipywidgets.Button(description='Check')
check_left_button = ipywidgets.Button(description='Check')
check_right_button = ipywidgets.Button(description='Check')
check_front_button = ipywidgets.Button(description='Check')
check_stop_button = ipywidgets.Button(description='Check')
check_back_button = ipywidgets.Button(description='Check')


def check_stop(change):
    global check_stop_widget
    PCA9685.set_channel_value(THROTTLE_CH, check_stop_widget.value)
def check_front(change):
    global check_front_widget
    PCA9685.set_channel_value(THROTTLE_CH, check_front_widget.value)
def check_back(change):
    global check_back_widget,check_stop_widget
    PCA9685.set_channel_value(THROTTLE_CH, check_stop_widget.value)
    time.sleep(0.1)
    PCA9685.set_channel_value(THROTTLE_CH, check_stop_widget.value)
    time.sleep(0.1)
    PCA9685.set_channel_value(THROTTLE_CH, check_back_widget.value)
def check_center(change):
    global check_center_widget
    PCA9685.set_channel_value(STEERING_CH, check_center_widget.value)
def check_left(change):
    global check_left_widget
    PCA9685.set_channel_value(STEERING_CH, check_left_widget.value)
def check_right(change):
    global check_right_widget
    PCA9685.set_channel_value(STEERING_CH, check_right_widget.value)
    
check_stop_button.on_click(check_stop)
check_front_button.on_click(check_front)
check_back_button.on_click(check_back)
check_center_button.on_click(check_center)
check_left_button.on_click(check_left)
check_right_button.on_click(check_right)
stop_button.on_click(check_stop)

In [13]:
check_final_widget = ipywidgets.VBox([
    ipywidgets.HBox([
        ipywidgets.VBox([check_left_widget,check_left_button]),
        ipywidgets.VBox([check_center_widget,check_center_button]),
        ipywidgets.VBox([check_right_widget,check_right_button])]),
    ipywidgets.HBox([
        ipywidgets.VBox([check_front_widget,check_front_button]),
        ipywidgets.VBox([check_stop_widget,check_stop_button]),
        ipywidgets.VBox([check_back_widget,check_back_button])]),
    stop_button
])

display(check_final_widget)